# Get metadata for instrument oil ("BZ=F")

https://github.com/chimpler/postgres-aws-s3/blob/master/README.md

https://medium.com/analytics-vidhya/data-flow-between-python-aws-s3-bucket-aws-rds-243e38561bc2

https://www.stackvidhya.com/write-a-file-to-s3-using-boto3/

## Requirements & configuration

In [1]:
# Import required packages
# import os
import pandas as pd
import yfinance as yf
import json
from pprint import pprint
import psycopg2
from sqlalchemy import create_engine
from datetime import date

In [2]:
# DB configuration
config = {
    'host': 'datalake.cknmu1bvrxjg.us-east-1.rds.amazonaws.com',
    'port': '5432',
    'user': 'muser',
    'password': 'datalake',
    'dbname': 'datalake',
}

# Configure cnx_string for sqlalchemy
cnx_str = f'postgresql://{config["user"]}:{config["password"]}@{config["host"]}/{config["dbname"]}'

In [4]:
# Set ticker for required instrument "Brent Crude Oil"
brent = yf.Ticker("BZ=F")

## Getting metadata of instrument 'BZ=F'

In [8]:
# Get metadata and convert it into csv format
dict_metadata = brent.info
df_metadata = pd.DataFrame(dict_metadata, index=[0]).T.reset_index()
df_metadata.rename(columns = {'index':'key', 0:'value'}, inplace = True)
df_metadata

,key,value
0,exchange,NYM
1,shortName,Brent Crude Oil Last Day Financ
2,exchangeTimezoneName,America/New_York
3,exchangeTimezoneShortName,EDT
4,isEsgPopulated,False
...,...,...
62,bidSize,0
63,dayHigh,106.2
64,regularMarketPrice,104.37
65,preMarketPrice,None


## Write metadata into datalake db

In [10]:
# Establish connection to database 'lakehouse'
try: 
    conn = psycopg2.connect(
        dbname=config['dbname'],
        user=config['user'],
        host=config['host'],
        password=config['password'],
        port=config['port']
    )

except psycopg2.Error as e: 
    print("Error: Could not make the connection to the postgres database")
    print(e)

# Create cursor
try: 
    cursor = conn.cursor()
except psycopg2.Error as e: 
    print("Error: Could not get the cursor to the database")
    print(e)

# Set auto commit feature
conn.set_session(autocommit=True)

# Create engine
engine = create_engine(cnx_str)

In [93]:
# Create table 'metadata_oil'
sql = """
    CREATE TABLE IF NOT EXISTS metadata_oil (
        key text,
        value text
        )
"""
cursor.execute(sql)

In [12]:
# Insert values into table using sqlalchemy
df_metadata.to_sql('metadata_oil', engine, if_exists='replace', index=False)

## Close connection

In [95]:
# Close connection
cursor.close()
conn.close()